## Advanced Computing Concepts with Dynamic Compute
In other tutorials we explored the fundementals of `Dynamic Compute`. In this notebook we will provide an overview of a subset of more complex examples of the API, such as:
* Interacting between `Mosaic`s and `ImageStack`s
* Interoperability between `DLTile` and other `AOI` objects
* More advanced batch-style, time series analysis examples

In [ ]:
import descarteslabs as dl
import descarteslabs.dynamic_compute as dc
from descarteslabs.dynamic_compute import Mosaic, ImageStack

First, we will set up some global variables and set up our input `ImageStack` and `Mosaic` objects. Here we will work with an `ImageStack` of `Sentinel-2` imagery corresponding to the date range of May to August, 2022 and a `Mosaic` of 2022's `Cropland Data Layer`:

In [ ]:
start_date = "2022-05-01"
end_date = "2022-08-01"
pid = "esa:sentinel-2:l2a:v1"
cdl_pid = "usda:cdl:v1"
bands = "nir red green blue"

In [ ]:
s2_stack = ImageStack.from_product_bands(
    pid, bands, start_datetime=start_date, end_datetime=end_date
).filter(lambda x: x.cloud_fraction < 0.2)

cdl_mosaic = Mosaic.from_product_bands(
    cdl_pid, "class", start_datetime="2022-01-01", end_datetime="2023-01-01"
)

### Interacting between ImageCollections and Mosaics
In the next few cells we will calculate NDVI through our time period and then mask to our Cropland Data Layer's Corn class:

In [ ]:
nir, red = s2_stack.unpack_bands("nir red")
ndvi = (nir - red) / (nir + red)

In [ ]:
ndvi_corn_mask = ndvi.mask(cdl_mosaic != 1)

In [ ]:
m = dc.map
m.center = 41.34232959809853, -95.54491138405865
m.zoom = 13

In [ ]:
_ = (
    s2_stack.pick_bands("red green blue")
    .median(axis="images")
    .visualize("Sentinel-2 Composite", m)
)
_ = ndvi.median(axis="images").visualize("NDVI Composite", m)
_ = cdl_mosaic.pick_bands("class").visualize("CDL", m, colormap="terrain")
_ = ndvi_corn_mask.median(axis="images").visualize(
    "NDVI Composite Corn Mask", m, colormap="magma"
)

In [ ]:
m

### Interoperability with other `GeoContext` Objects
Now that we have constructed the base Image layer for our analysis we can extract information from and summarize our results over any type of geometry. In the next few cells we will demonstrate how you can `compute` over `shapely geometries` or `DLTile`s:

In [ ]:
from shapely.wkt import loads
import matplotlib.pyplot as plt

You can create and `compute` over an `AOI` generated from a `shapely Polygon`:

In [ ]:
geom = loads(
    """POLYGON ((-95.54491138405865 41.34232959809853, 
    -95.52455234632363 41.34232959809853, 
    -95.52455234632363 41.35521625255075, 
    -95.54491138405865 41.35521625255075, 
    -95.54491138405865 41.34232959809853))"""
)
aoi = dl.geo.AOI(geom, resolution=10.0, crs="EPSG:3857")

`Compute` median masked NDVI through our time period over our `AOI`:

In [ ]:
ndvi_mask_arr = ndvi_corn_mask.median(axis="images").compute(aoi).ndarray
dl.utils.display(ndvi_mask_arr[0, :, :], colormap="viridis", figsize=(5, 5))

And also over `DLTile` objects:

In [ ]:
dltile = dl.geo.DLTile.from_latlon(
    41.34232959809853, -95.54491138405865, tilesize=512, pad=0, resolution=10.0
)

In [ ]:
ndvi_mask_arr = ndvi_corn_mask.median(axis="images").compute(dltile).ndarray
dl.utils.display(ndvi_mask_arr, colormap="viridis", figsize=(5, 5))

### Time Series Analysis and Reduction with `ImageStacks`
In the following cells we will demonstrate how you can utilize an `ImageStack` to retrieve time series summary statistics and `ndarrays`. Note here we will start with our input `ImageStack`, which has already been masked by our CDL `Mosaic`, and all resulting objects from our aggregation are `Mosaic` objects. 

* `axis='pixels'`
    * First, we'll call `.compute` over our `ImageStack` along `axis='pixels'` to calculate the median value _across each `GeoContext`_ for each `Image`. When calling `axis='pixels'` we are _aggregating the spatial dimension to a single value_.
* `axis='images'`
    * Next we will explore calling `.compute` over an `ImageStack` along the `axis='images'` to build simple data composites. When computing over the `images` axis we will be aggregating all `Images` in our `ImageStack` into a single `Image` equivalent.

First we will call `propterties.compute` over our `ImageStack` to retrieve the stack's pertinent information, such as `datetime` and `Image ID`, which we will keep track of for later:

In [ ]:
ndvi_props = ndvi_corn_mask.properties.compute(dltile)
ndvi_props[0]

In [ ]:
ndvi_dates = [p["acquired"].strftime("%Y-%m-%d") for p in ndvi_props]

ndvi_ids = [p["id"] for p in ndvi_props]

Next we will explore `axis='pixels'`, where the returned value is a single statistic corresponding to an `Image`'s acquired date:

In [ ]:
ndvi_median = ndvi_corn_mask.median(axis="pixels").compute(dltile)
ndvi_median_list = ndvi_median.ndarray[:, 0].tolist()
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ndvi_dates, ndvi_median_list)

Comparing `axis='pixels'` to `axis='images'`, we'll see the resulting `ndarray` is the same shape as that of a single-dimensional `Mosaic`, in this case each pixel represents the median value throughout the time period:

In [ ]:
ndvi_median_arr = ndvi_corn_mask.median(axis="images").compute(dltile).ndarray
dl.utils.display(ndvi_median_arr, colormap="viridis", figsize=(5, 5))

Lastly we will plot out the underlying `ImageStack` with it's associated NDVI, Acquired Date, and Unique ID:

In [ ]:
title_list = []
for i in range(len(ndvi_dates)):
    title_list.append(
        f"Mean NDVI: {ndvi_median_list[i]}\nAcquired Date: {ndvi_dates[i]}\nImage ID: {ndvi_ids[0]}"
    )

In [ ]:
ndvi_stack = ndvi_corn_mask.compute(dltile)
dl.utils.display(*ndvi_stack.ndarray, title=title_list, colormap="viridis")